# Section: Encrypted Deep Learning


# Encrypted Computations in PySyft

In [2]:
import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

W0711 21:44:38.286935 140259911964544 hook.py:98] Torch was already hooked... skipping hooking process


In [0]:
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

In [0]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

In [0]:
# we assigned one of the parties, secure_worker, to generate the random numbers 
# we have trust that the secure_worker, one of the parties, does not know Alice or Bob (they also do not know each other)
# use the parameter crypto_provider to assign on of the parties

x = x.share(bob, alice, crypto_provider=secure_worker)

In [0]:
y = y.share(bob, alice, crypto_provider=secure_worker) # notice the number of the parties where the secrete is shared

In [6]:
bob._objects

{5832710301: tensor([1942124677794672934, 1386511262076046253, 1864651451166692784,
         2923358307548933517]),
 37155951963: tensor([1024298226974157009, 3757621307906408434, 3212090408686283635,
          585039451028585162])}

We have shared two secretes with three parties. Let's try some computations remotely on these secretes:

In [7]:
# adding secretes on remote machines
z = x + y
z.get() # returns and decodes

tensor([3, 1, 4, 4])

In [8]:
# subtracting secretes on remote machines

z = x - y
z.get()

tensor([-1,  3,  2,  4])

In [9]:
# multiplying secretes on remote machines

z = x * y
z.get()

tensor([ 2, -2,  3,  0])

In [10]:
# boolean operations on remote machines
z = x > y
z.get()

tensor([0, 1, 1, 1])

In [11]:
z = x < y
z.get()

tensor([1, 0, 0, 0])

In [12]:
z = x == y
z.get()

tensor([0, 0, 0, 0])

In [0]:
# double check that your implementation is using the fix_precision() function when dealing with float values
# reverse this function using the float_precision() function

x = th.tensor([1.2, 2.2, 3.2, 4.2])
y = th.tensor([2.1, -1.1, 1.1, 0.1])

x = x.fix_precision().share(bob, alice, crypto_provider=secure_worker)
y = y.fix_precision().share(bob, alice, crypto_provider=secure_worker)

In [15]:
z = x + y
z.get().float_precision()

tensor([3.3000, 1.1000, 4.3000, 4.3000])

In [16]:
z = x - y
z.get().float_precision()

tensor([-0.9000,  3.3000,  2.1000,  4.1000])

In [17]:
z = x * y
z.get().float_precision()

tensor([ 2.5200, -2.4200,  3.5200,  0.4200])

In [18]:
z = x > y
z.get().float_precision()

tensor([0., 1., 1., 1.])

In [19]:
z = x < y
z.get().float_precision()

tensor([1., 0., 0., 0.])

In [20]:
z = x == x
z.get().float_precision()

tensor([1., 1., 1., 1.])

# Lesson: Encrypted Deep Learning in PyTorch

### Build your algorithms and Model

In [0]:
from torch import nn
from torch import optim
import torch.nn.functional as F

# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x



def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)
        


In [13]:
model = Net()

#train the model
train()

tensor(2.1571)
tensor(12.9535)
tensor(21.6353)
tensor(1.1473)
tensor(0.9781)
tensor(0.9570)
tensor(0.9398)
tensor(0.9207)
tensor(0.8990)
tensor(0.8741)
tensor(0.8455)
tensor(0.8128)
tensor(0.7757)
tensor(0.7359)
tensor(0.6929)
tensor(0.6448)
tensor(0.5935)
tensor(0.5378)
tensor(0.4791)
tensor(0.4152)


In [14]:
# run predictions
model(data)

tensor([[0.1881],
        [0.4219],
        [0.8541],
        [0.6796]], grad_fn=<AddmmBackward>)

## Encrypt the Model and Data

In [15]:
encrypted_model = model.fix_precision().share(alice, bob, crypto_provider=secure_worker)
encrypted_model

Net(
  (fc1): Linear(in_features=2, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=1, bias=True)
)

In [8]:
list(encrypted_model.parameters())

[Parameter containing:
 Parameter>AutogradTensor>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:5747582235 -> bob:22955187796]
 	-> (Wrapper)>[PointerTensor | me:70178304243 -> alice:68052171563]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>AutogradTensor>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:43107561652 -> bob:71961852178]
 	-> (Wrapper)>[PointerTensor | me:86414278975 -> alice:68762703899]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>AutogradTensor>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:50688519605 -> bob:86048180237]
 	-> (Wrapper)>[PointerTensor | me:55780423456 -> alice:63836427461]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>AutogradTensor>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:23107485845 -> bob:14352544615]
 	-> 

In [0]:
encrypted_data = data.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [0]:
encrypted_prediction = encrypted_model(encrypted_data)

In [18]:
encrypted_prediction.get().float_precision()

tensor([[0.1870],
        [0.4210],
        [0.8530],
        [0.6780]])

# Reuse the MNIST NN from the previous classes (firs week of classes) to train the classifier with a Secure Federated learning appraoch.